### 인덱스 설계

- 행 개수가 100만 건 이상인 테이블 4개(주문, 주문상세, 결제, 배송)를 대상으로 인덱스 설계 수행

In [1]:
from dotenv import load_dotenv
import os
import psycopg2
import io
import pandas as pd
import time

In [2]:
load_dotenv('.env')
db_name = os.getenv('db_name')
user_name = os.getenv('db_admin')
password = os.getenv('password')

conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database=db_name,
    user=user_name,
    password=password
)

cur = conn.cursor()
cur.execute("SELECT current_database();")
print(cur.fetchone())

('saturnbird',)


#### 1. 결제 테이블 인덱스 설계

In [3]:
cur.execute('''EXPLAIN ANALYZE
SELECT SUM(총상품금액) AS 일매출
FROM 결제
WHERE 결제상태 = '결제완료'
AND 결제날짜 = CURRENT_DATE;''')

for i in cur.fetchall():
    print(i)

('Finalize Aggregate  (cost=58013.61..58013.62 rows=1 width=8) (actual time=212.237..216.567 rows=1 loops=1)',)
('  ->  Gather  (cost=58013.40..58013.61 rows=2 width=8) (actual time=212.103..216.561 rows=3 loops=1)',)
('        Workers Planned: 2',)
('        Workers Launched: 2',)
('        ->  Partial Aggregate  (cost=57013.40..57013.41 rows=1 width=8) (actual time=183.749..183.750 rows=1 loops=3)',)
('              ->  Parallel Seq Scan on "결제"  (cost=0.00..57004.85 rows=3420 width=4) (actual time=183.747..183.747 rows=0 loops=3)',)
('                    Filter: ((("결제상태")::text = \'결제완료\'::text) AND ("결제날짜" = CURRENT_DATE))',)
('                    Rows Removed by Filter: 999993',)
('Planning Time: 1.858 ms',)
('Execution Time: 216.631 ms',)


In [4]:
cur.execute('''CREATE INDEX 결제테이블_idx
ON 결제(결제상태, 결제날짜);''')

In [ ]:
# 일 매출 설계
cur.execute('''EXPLAIN ANALYZE
SELECT SUM(총상품금액) AS 일매출
FROM 결제
WHERE 결제상태 = '결제완료'
AND 결제날짜 = CURRENT_DATE;''')

for i in cur.fetchall():
    print(i)

('Aggregate  (cost=13135.86..13135.87 rows=1 width=8) (actual time=0.039..0.040 rows=1 loops=1)',)
('  ->  Index Scan using "결제테이블_idx" on "결제"  (cost=0.43..13115.34 rows=8208 width=4) (actual time=0.035..0.036 rows=0 loops=1)',)
('        Index Cond: ((("결제상태")::text = \'결제완료\'::text) AND ("결제날짜" = CURRENT_DATE))',)
('Planning Time: 0.338 ms',)
('Execution Time: 0.096 ms',)


In [ ]:
cur.execute('''EXPLAIN ANALYZE
SELECT SUM(총상품금액) AS 일매출
FROM 결제
WHERE 결제상태 = '결제완료'
AND 결제날짜 = CURRENT_DATE;''')

for i in cur.fetchall():
    print(i)